# Deploy




![](https://carabedo.github.io/apis/http.png)

# Flask:

Nos permite 'apificar' nuestro modelo, en realidad cualquier script de python. 

Podemos enviar un GET requests a algun puerto de nuestra computadora. 


Ese requests puede contener los parametros que luego el script va usar.

###  GET Method


GET es el metodo HTTP mas comun, habran notado que navegando por la internet los siguiente:

>www.ejemplo.com/demo_form.php?name1=value1&name2=value2

En la URL del request estan los parametros del pedido.

* GET requests can be cached
* GET requests remain in the browser history
* GET requests can be bookmarked
* GET requests should never be used when dealing with sensitive data
* GET requests have length restrictions
* GET requests are only used to request data (not modify)

En este es una simple app que imprime el cuadrado de la variable input 'a':

```python
from flask import Flask, jsonify, request
app = Flask('Servidor Get') #nombre de la app
@app.route('/',methods=['GET']) #metodo http que va a usar
def funcionprincipal():#no importa el nombre el decorador se encarga
    # obtengo los datos del request que viene de la peticion externa 
    data=request.args.to_dict() # (el metodo to_dict lo transforma en un diccionario)
    try:# uso try por si no me envian una variable a           
        resp='el cuadrado de a es : '+str(int(data['a'])*int(data['a']))
    except:
        resp='no se envio la variable a'
    return(resp)
app.run(host='127.0.0.1',  port=5002 )# host local puerto el que quieras
```

Para que funcione debemos guardar el codigo en un archivo 'prueba.py' 

y luego correrlo en la consola `python prueba.py`.



* http://127.0.0.1:5002/?a=4

* http://127.0.0.1:5002/?a=4&b='asdfasf'

* http://127.0.0.1:5002/?c=4&b='asdfasf'

Y si quisiera pasarle un vector?

* http://127.0.0.1:5002/?a=['1','12']



###  POST Method

**el metodo POST se usa para enviar informacion al servidor para crear o actualizar algun recurso.**

>en esta caso la  data enviada al servidor con POST viaja en el cuerpo del request:

```
POST /test/demo_form.php HTTP/1.1
Host: w3schools.com
name1=value1&name2=value2
POST is one of the most common HTTP methods.
```

* POST requests are never cached
* POST requests do not remain in the browser history
* POST requests cannot be bookmarked
* POST requests have no restrictions on data length
* Los requests POST no se pueden hacer desde un navegador

In [ ]:
from flask import Flask, jsonify, request
app = Flask('server post')
@app.route('/',methods=['POST']) #aca definimos q recibe requests POST
def predict(): #la funcion q se ejecuta 
    data = request.get_json(force=True)
    try:
        a_vector = np.array(data['a']).astype('int')
        # Le damos forma de un diccionario para poder hacer el traspaso a json trivialmente
        a_2=a_vector**2    
        resp={'response' : a_2.tolist() }#importante pasar a lista los numpy arrays 
    except:
        resp={'response' : 'no esta presente la variable a'}
        # en esta linea, transformamos el diccionario en json con jsonify (funcionalidad de flask)
        # este json es incorporado en el cuerpo de la respuesta
    return jsonify(resp)
app.run(host='127.0.0.1', port=5001)

Queremos pasar la variable 'a' como un vector, con el metodo POST es muy sencillo.

In [44]:
import requests as req
import json
# parametros, la url y un diccionario
url='http://127.0.0.1:5001/' # <--- url por default en servidor
data = { 'a': ['1', '-2', '4', '13', '5', '0', '10', '1'] , 'b' : 2342 }
# el requests
r = req.post(url, json=data)
r

<Response [200]>

In [45]:
r.json()

{'response': [1, 4, 16, 169, 25, 0, 100, 1]}

In [46]:
r.json()['response']

[1, 4, 16, 169, 25, 0, 100, 1]

In [47]:
type(r.json()['response'])

list

In [158]:
from flask import Flask, jsonify, request
app = Flask('Nombre')
@app.route('/',methods=['POST'])
#defino la funcion que manejarara el request
def predict():
    # obtengo los datos del request post
    data = request.get_json(force=True)
    ###
    ### cositas lindas
    ###
    return jsonify(resp)

app.run(host='127.0.0.1', port=5001)



# Debugeando

Este modo nos permite que la app se ejecute nuevamente cada vez que el archivo `miapp.py` sea modificado. Nos permite poder ir cambiando el codigo si es que algo no funciona bien, ayuda poner algunos `print` en el codigo lo que se vera en la consola donde este corriendo la app.

In [158]:
from flask import Flask, jsonify, request
app = Flask('Nombre')
@app.route('/',methods=['POST'])
def predict():
    data = request.get_json(force=True)
    return jsonify(resp)
app.run(host='127.0.0.1', port=5001, debug=True)



# Ok... algo un poco mas interactivo, no hay?

Esta app de Flask genera una pagina con un grafico de Plotly con el cual podemos interactuar!
Vamos a armar un grafico con los datos de contagios de covid-19 para diferentes paises. Usamos requests GET asi el usuario puede mandar la informacion del pais del cual quiere ver los datos en la URL




Local:

*   http://127.0.0.1:5000/?paises=['AR','FR','ES']&tipo='muertes_100k'
*   http://127.0.0.1:5000/?paises=['AR','FR','ES']&tipo='casos_100k'

Externo:

*   http://127.0.0.1:5000/?paises=['AR','FR','ES']&tipo='muertes_100k'


https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2

```python
from flask import Flask, render_template
from flask import jsonify, request
import pandas as pd
import json
import plotly
import plotly.express as px
import requests as req
import numpy as np
```

```python
app = Flask(__name__)
@app.route('/',methods=['GET'])
def notdash():
    params=request.args.to_dict()
    paises=json.loads(params['paises'].replace('\'', '"'))
    tipo=json.loads(params['tipo'].replace('\'', '"'))
    fig = px.line(title=tipo)
    for pais in paises:
        r=req.get('http://corona-api.com/countries/' + pais)
        t=[]
        casos=[]
        muertes=[]
        data=r.json()
        for day in data['data']['timeline']:
                t.append(day['date'])
                casos.append(day['new_confirmed'])
                muertes.append(day['new_deaths'])
        df=pd.DataFrame()
        df.index=pd.to_datetime(t)
        df['casos']=casos
        df['muertes']=muertes
        pop=data['data']['population']/100000
        df['casos_100k']=np.array(casos)/pop
        df['muertes_100k']=np.array(muertes)/pop
        fig.add_scatter(x=df.index, y=df[tipo],name=pais,mode='markers+lines') 
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    return render_template('notdash.html', graphJSON=graphJSON)
app.run(debug=True)
```

## Y dash? Eso que plotly me quiere vender todo el tiempo?


https://dash.gallery/Portal/?search=[ML]

![](https://images.prismic.io/plotly-marketing-website/e9302a4a-b71c-4061-b28b-f69a6ecbb041_Dash_autonomous.gif)


# Streamlit

* Flask tiene la gran desventaja que solo se puede interactuar mediante requests.

* Streamlit es un framework para crear web-apps de una manera super sencilla y dinámica.

* Todos los elementos de una página web son definidos en un **script de python**. Cada uno será definido por una función y a medida que los agregamos podemos ver su resultado en una página de nuestro navegador.


![](https://raw.githubusercontent.com/streamlit/docs/main/public/images/Streamlit_overview.gif)




<video width="720"  controls>
  <source src="https://s3-us-west-2.amazonaws.com/assets.streamlit.io/videos/hero-video.mp4" type="video/mp4">
</video>

```python
import streamlit as st

x=st.slider('eleji valor')

st.write('el cubo de ',x , ' es', x*x*x)
```

![](1.png)

https://www.streamlit.io/


* Las apps de Streamlit son scripts de python que se ejecutan secuencialmente. 

* Cada vez que un usuario accede a la app, el script se re ejecuta.

* Duran la ejeccion, Streamlit muestra el output en el navegador.

* Cada interaccion con un widget hace que el script se re-ejecute con el nuevo valor del widget.

* Se puede usar **cache** para evitar recalcular funciones muy pesadas, mejorando el tiempo de respuesta de la app.


## widgets

Los widgets son objetos que sirve para que el usuario interactue con los datos o el modelo, hay una extensa lista:

https://docs.streamlit.io/en/stable/api.html

Ejemplo:

```python
import streamlit as st
x = st.slider('x')  # 👈 widget
st.write(x, 'squared is', x * x)
```

En este ejmeplo la app tira un output “0 squared is 0”. Cada vez que el usuario interactua , Streamlit vuelve a ejecutar el script entero, asignando el nuevo valor a la variable asociadad al widget. Si fuera 10, Streamlit corre el codigo y daria como output “10 squared is 100”.

## componentes 

widgets developeados por la comunidad que extienden la funcionalidad de Streamlit.

https://www.streamlit.io/components

```python
import streamlit as st
import pandas as pd 
import numpy as np
import plotly.express as px
import requests as req

paises = st.multiselect(
     'Paises a visualizar:',
     ['AR', 'BR', 'CH', 'UY','BO'])


# texto
option = st.selectbox(
     'Que datos desea visualizar?',
     ('casos_100k','muertes_100k'))

tipo=option

```

```python
if st.button('plot'):
     fig = px.line(title=tipo)
     for pais in paises:
          r=req.get('http://corona-api.com/countries/' + pais)
          t=[]
          casos=[]
          muertes=[]
          data=r.json()
          for day in data['data']['timeline']:
                    t.append(day['date'])
                    casos.append(day['new_confirmed'])
                    muertes.append(day['new_deaths'])
          df=pd.DataFrame()
          df.index=pd.to_datetime(t)
          df['casos']=casos
          df['muertes']=muertes
          pop=data['data']['population']/100000
          df['casos_100k']=np.array(casos)/pop
          df['muertes_100k']=np.array(muertes)/pop
          fig.add_scatter(x=df.index, y=df[tipo],name=pais,mode='markers+lines') 
      st.plotly_chart(fig)
else:
     pass
```

Guardamos todo el codigo en un archivo ".py" y luego lo ejecutamos desde la consola:
    
```shell
streamlit run miarchivito.py
```


http://192.168.0.18:8501

### Muy bonito! pero como se lo muestro al cliente/jefa/mivieja? 


* Firewalls

* Llamar a fibertel para que abran puertos o habiliten una DMZ

* Peligroso

* Siempre tiene que estar corriendo la terminal de python



# Cloud computing

![](2.png)

![](3.png)

![](7.png)

![](8.png)


![](9.png)


<img src="5.png" style="width: 500px;"/>

[Repo](https://github.com/carabedo/properatti)
[App](https://geopami.herokuapp.com/)

# IAAS (infrastructure-as-a-service)


![](iaas.jpg)




## Virtual Machines


Ejemplo en oracle:

Creamos una vm [aca](https://cloud.oracle.com/compute/instances?region=sa-vinhedo-1)





# como controlamos la vm?


![](https://i.giphy.com/media/YQitE4YNQNahy/giphy.webp)

## ssh


Le cambiamos los permisos a la llave que nos bajamos:

`chmod 400 <private_key_file>`

En mi caso:

`ssh -i sshtest.key` 

Nos conectamos a una terminal de la maquina virtual usando la llave:

`ssh -i <private_key_file> <username>@<public-ip-address>`

En mi caso:

` ssh -i sshtest.key opc@193.123.101.232`      

[tutorial de oracle](https://docs.oracle.com/en-us/iaas/Content/Compute/Tasks/accessinginstance.htm)

## ssh

Creemos una carpeta:

`mkdir prueba`

Entremos en la carpeta y creemos un archivo con codigo en python

```
    cd prueba
    vim prueba.py
```

Ejecutemos nuestro script:

`python3 prueba.py`

### Subiendo archivos

Desde una consola local usamos el programita scp (secure copy):

`scp -i key.pem -r /path/local/files root@:0.0.0.0:/path/on/my/server`


Necitamos la ip publica del servidor, en nuestro caso quedaria asi:


`scp -i sshtest.key  ./server_bokeh.py opc@193.123.101.232:/home/opc/prueba`  


Esto subiria el archivo `server_bokeh.py` a la maquina virtual a una carpeta llamada `prueba`. Solo funciona si ejecuto el comando en la misma carpeta donde se encuentra dicho archivo y la llave.


### Subiendo carpetas

* -r recursivamente la carpeta templates

`scp -i sshtest.key  -r ./templates/ opc@193.123.101.232:/home/opc/prueba`   

## ssh con visualcode

<img src='https://code.visualstudio.com/assets/docs/remote/ssh-tutorial/remote-ssh-extension.png' width='500' />



<img src='https://code.visualstudio.com/assets/docs/remote/ssh-tutorial/remote-commands.png' width='500' />

Hay que editar el archivo que guarda la direccion de las maquinas virtuales y la ubicacion en nuestra compu de las llaves. En Host ponemos el nombre que queremos, HostName es la ip de la maquina, luego el path de la .key en nuestra compu y por ultimo el nombre del usuario en la maquina virtual para las vms de oracle es `opc`.

```
Host Oracle
  HostName 193.123.110.190
  IdentityFile /Users/fernando/gits/deploy/sshtest.key
  User opc
```

No importa en que plataforma tenemos nuestra maquina virtual, solo necesitamos la ip publica, una llave y saber el nombre de nuestro usuario.

### Instalamos conda: 

es una VM vacia, necesitamos instalar todas las librerias que vamos a usar

`wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh`

`sh Miniconda3-latest-Linux-x86_64.sh`

# flask en una vm

* levantemos un servidor de flask en nuestra vm, guardamos el codigo en `app.py`.

`flask run --host=0.0.0.0`

* abrimos el puerto donde esta flask:

`sudo iptables -I INPUT 6 -m state --state NEW -p tcp --dport 5000 -j ACCEPT`

* agregamos una `Ingress Rules` en la `Security List` de la subnet asociada a nuestra VM desde la pagina de oracle.


ahora nuestra 'app' es accesible desde cualquier maquina




* http://193.123.110.190:5002/?a=4





# flask+plotly en una vm

* levantemos un servidor de flask+plotly en nuestra vm.

`flask run --host=0.0.0.0`

* solo podemos tener una app por puerto 

ahora nuestra 'app' es accesible desde cualquier maquina




* http://193.123.110.190:5000/?paises=['AR','FR','ES']&tipo='muertes_100k'





# screen

un programa que nos permite cerrar la consola sin que nuestro servidor deje de funcionar.

`sudo yum install screen`

iniciamos la sesion 

`screen`

abrimos el servidor

`detach from the session using Clrl+A and then press d.`

`screen -ls` vemos las sessiones abiertas

`screen -r ID` nos volvemos a conectar



Tambien podemos crear sessiones con nombres mas faciles de recordar

`screen -S plotlyserver`


`screen -r  plotlyserver` 



# Serverless
![](6.png)

# lambdas (aws)

### api gateway
### layers

https://github.com/keithrozario/Klayers

### test&deploy

```python
import json
import numpy as np

def lambda_handler(event, context):
    body = json.loads(event['body'])
    x=np.array(body['x'])
    y=x**2 
    result=[{'body' : body, 'y' : y.tolist(), 'name' : 'cuadratica'}]
    return {
        'statusCode': 200,
        'body': json.dumps(result)
    }
```


In [50]:
import numpy as np
data={"x": np.array([1,2,3,4]).tolist()}
respuesta=req.post('https://6xpc1u0n12.execute-api.us-east-2.amazonaws.com/default/test2',json=data)
respuesta

<Response [200]>

In [51]:
respuesta.json()

[{'body': {'x': [1, 2, 3, 4]}, 'y': [1, 4, 9, 16], 'name': 'cuadratica'}]

In [52]:
type(respuesta.json()[0]['y']),type(respuesta.json()[0]['y'][0])

(list, int)

In [53]:
import pandas as pd

df=pd.read_csv('https://raw.githubusercontent.com/carabedo/labs/master/data/freddo.csv')
df

,temp,venta
0,12.2,125
1,14.4,175
2,15.0,325
3,16.7,275
4,17.8,425
5,18.3,400
6,19.4,450
7,22.2,425
8,22.8,450
9,23.9,525


In [54]:
data={'x': df.values.tolist()}
respuesta=req.post('https://6xpc1u0n12.execute-api.us-east-2.amazonaws.com/default/test2',json=data)
respuesta

<Response [200]>

In [55]:
respuesta.json()[0]['y']

[[148.83999999999997, 15625.0],
 [207.36, 30625.0],
 [225.0, 105625.0],
 [278.89, 75625.0],
 [316.84000000000003, 180625.0],
 [334.89000000000004, 160000.0],
 [376.35999999999996, 202500.0],
 [492.84, 180625.0],
 [519.84, 202500.0],
 [571.2099999999999, 275625.0],
 [625.0, 302500.0],
 [712.89, 390625.0]]

In [56]:
type(respuesta.json()[0]['y']),type(respuesta.json()[0]['y'][0]),type(respuesta.json()[0]['y'][0][0])

(list, list, float)

Links
---

### setear flask en una vm:

https://developer.oracle.com/oracle-cloud-infrastructure/compute-vm-simple-tutorial/
https://docs.oracle.com/es-ww/iaas/developer-tutorials/tutorials/flask-on-ubuntu/01oci-ubuntu-flask-summary.html

### instalar conda en oracle linux 7

https://deeplearning.lipingyang.org/2018/12/24/install-miniconda-on-centos-7-redhat-7/